## Pre-trained models

### (1) VGG-16

In [88]:
import io
import torch
from PIL import Image
import requests
from torch.autograd import Variable
import torchvision.models as models
import torchvision.transforms as transforms

# You can get all the pre-trained models here: https://pytorch.org/vision/stable/models.html
vgg16 = models.vgg16(pretrained=True)  # This may take a few minutes.

In [97]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [92]:
# Read the categories
# Download ImageNet labels
# https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

with open("/Downloads/imagenet_classes.txt", "r") as f:
    labels = [s.strip() for s in f.readlines()]

In [98]:
import urllib
#url, filename = ("https://github.com/dataman-git/codes_for_articles/blob/master/pic/tesla.png?raw=true", "tesla.jpg")
url, filename = ("https://cff2.earth.com/uploads/2022/01/06080341/Goldfish.jpg?raw=true", "goldfish.jpg")
url, filename = ("https://cdn.britannica.com/92/152292-050-EAF28A45/Bald-eagle.jpg?raw=true", "eagle.jpg")

try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename).convert('RGB')
print(input_image.size)
input_image.show()

#pix = np.array(im.getdata()).reshape(im.size[0], im.size[1], 3)
#pix[1:10]

(1600, 1071)


In [94]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image) 
input_tensor.shape # = torch.Size([3, 224, 224])
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
input_tensor.shape # = torch.Size([1, 3, 224, 224])

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

In [95]:
output = vgg16(input_batch)  # Returns a Tensor of shape (batch, num class labels)
#prediction = prediction.data.numpy().argmax()  # Our prediction will be the index of the class label with the largest value.
#prediction

# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
#print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
#print(probabilities)

In [96]:
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(labels[top5_catid[i]], top5_prob[i].item())

bald eagle 0.9969350099563599
vulture 0.0015524202026426792
kite 0.001502618077211082
albatross 3.8109526485641254e-06
hornbill 1.904312739497982e-06
